# **The Hough Transform [Part 2]**

**Recommended Reading/Viewing:**

1. [How the Hough Transform was Invented](http://www.rci.rutgers.edu/~shunsun/resource/Hough_Transform.pdf)

**Additional Reading**
1. Placeholder

**Recommended [Jupyter Theme](https://github.com/dunovank/jupyter-themes) for presenting this notebook:**
````
jt -t grade3 -cellw=90% -fs=20 -tfs=20 -ofs=20
````

---

Last time we left off with two questions:

![](../videos/hough_two.gif)

## 1. How can we use the Hough Transform to **efficiently** find **noisy** colinear points?
## 2. How might we modify the Hough Transform to avoid having intersection points at $\infty$?

![](../graphics/spacer_small.png)

- To anwers these questions, we need to visit SRI (Stanford Research Institute) in the late 1960s. 
- This was an interesting time at SRI. Researchers were building the **world's first mobile robot,** [**Shakey**](https://www.youtube.com/watch?v=GmU7SimFkpU).

![](../videos/shakey_1.gif)

- One of the many problems SRI researchers faced in developing Shakey was creating a vision system that would allow Shakey to interpet the "Blocks World" he operated in.

![](../videos/shakey_2.gif)

- SRI Researchers **Peter E. Hart** and **Richard O. Duda** were looking for an efficient and reliable method for detecting lines in images. 
- Hart + Duda likely used **Robert's Cross** or the **Sobel-Feldman Operator** to find edge pixels in images. 
- They were then trying to use these images to "build a model of the local environment and to update the position of the robot from identificaion of known landmarks."

![](../graphics/hart_and_duda-01.png)

- In search for a solution, **Peter Hart** studied one of the few computer vision books at the time, **Azriel Rosenfeld's Picture Processing by Computer.**
- In this book, Rosenfeld briefly mentions the then obscure Hough patent, describing the tranformation algabraically for the first time, and posed a simple and computationally efficient method of implementing Hough's method. 
- However, Rosenfeld does **not solve the intersection at $\infty$ problem.**
- Simultaneously, Hart was examining another appraoch that he thought may be helpful in pattern recognition - **integral geometry.** This approach didn't work, but he did pick up a potentially useful alternative parameterization of a line.

[Vertical Hough line video placeholder]